In [1]:
import requests
import json
import datetime
from pprint import pprint
import pandas as pd
import numpy as np

In [2]:
#ローカルにおいてあるconfigファイルを読み出して、認証情報を取得しておく
import sys
sys.path.append("../")
from instaapi.local_config import * 

In [3]:
def basic_info():
    # 初期
    config = dict()
    #アクセストークン※facebook開発アプリから取得
    config["access_token"]         = ACCESS_TOKEN
    #アプリID※facebook開発アプリから取得
    config["app_id"]               = APP_ID
    #アプリシークレット※facebook開発アプリから取得
    config["app_secret"]           = APP_SECRET
    #インスタグラムビジネスアカウントID※facebook開発アプリから取得
    config['instagram_account_id'] = INSTAGRAM_ACCOUNT_ID
    #APIバージョン
    config["version"]              = 'v16.0'
    # 【修正不要】graphドメイン
    config["graph_domain"]         = 'https://graph.facebook.com/'
    # 【修正不要】エンドポイント
    config["endpoint_base"]        = config["graph_domain"]+config["version"] + '/'
    # 出力
    return config

In [4]:
# リクエスト
params   = basic_info()       # リクエストパラメータ
#response = debugAT(params)    # レスポンス


In [5]:
'''各投稿のinformation(基本的な情報)を取得'''


import requests
#import pandas as pd
#pd.set_option('display.max_rows', None)

# アクセス情報
business_account_id = INSTAGRAM_ACCOUNT_ID
token = ACCESS_TOKEN
username = USER_NAME
fields = 'name,username,biography,follows_count,followers_count,media_count'
media_fields = 'timestamp,permalink,media_url,like_count,comments_count'
period = 'day'


# ユーザー情報を取得する
def user_info(business_account_id=INSTAGRAM_ACCOUNT_ID,token=ACCESS_TOKEN,username=username,fields=fields):
    request_url = "https://graph.facebook.com/v16.0/{business_account_id}?fields=business_discovery.username({username}){{{fields}}}&access_token={token}".format(business_account_id=business_account_id,username=username,fields=fields,token=token)
#     print(request_url)
    response = requests.get(request_url)
    return response.json()['business_discovery']

#print(user_info(business_account_id,token,username,fields))

# メディア情報を取得する
def user_media_info(business_account_id,token,username,media_fields):
    all_response = []

    request_url = "https://graph.facebook.com/v16.0/{business_account_id}?fields=business_discovery.username({username}){{media{{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token)
#     print(request_url)
    response = requests.get(request_url)
    result = response.json()['business_discovery']
    
    all_response.append(result['media']['data'])
    
    # 過去分がある場合は過去分全て取得する(1度に取得できる件数は25件)
    if 'after' in result['media']['paging']['cursors'].keys():
        next_token = result['media']['paging']['cursors']['after']
        while next_token is not None:
            request_url = "https://graph.facebook.com/v16.0/{business_account_id}?fields=business_discovery.username({username}){{media.after({next_token}){{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token,next_token=next_token)
#             print(request_url)
            response = requests.get(request_url)
            result = response.json()['business_discovery']
            all_response.append(result['media']['data'])
            if 'after' in result['media']['paging']['cursors'].keys():
                next_token = result['media']['paging']['cursors']['after']
            else:
                next_token = None
    
    return all_response

result = user_media_info(business_account_id,token,username,media_fields)

'''結果をデータフレームに格納'''
df_media_info = pd.DataFrame(result[0])
for noc in np.arange(1, len(result)):
    output_per_call = pd.DataFrame(result[noc])
    df_media_info = pd.concat([df_media_info,output_per_call], ignore_index=True)
    
df_media_info


,timestamp,permalink,media_url,like_count,comments_count,id
0,2023-05-03T09:30:00+0000,https://www.instagram.com/p/CrvFzVnr1qb/,https://scontent.cdninstagram.com/v/t51.29350-...,59,1,17992966069785815
1,2023-05-02T09:30:00+0000,https://www.instagram.com/p/Cr0PXL2LIdk/,https://scontent.cdninstagram.com/v/t51.29350-...,49,0,17974722560215754
2,2023-05-01T09:30:00+0000,https://www.instagram.com/p/CrshBJzr1g9/,https://scontent.cdninstagram.com/v/t51.29350-...,56,0,17998017016835519
3,2023-04-27T09:30:00+0000,https://www.instagram.com/p/CrkymADrCAZ/,https://scontent.cdninstagram.com/v/t51.29350-...,80,0,18356028925053534
4,2023-04-26T09:30:00+0000,https://www.instagram.com/p/CrfpDdxLtal/,https://scontent.cdninstagram.com/v/t51.29350-...,63,0,18056555143403921
...,...,...,...,...,...,...
111,2022-01-13T13:16:58+0000,https://www.instagram.com/p/CYq-71wpmv4/,https://scontent.cdninstagram.com/v/t51.29350-...,127,0,17906032391509098
112,2021-12-07T12:01:02+0000,https://www.instagram.com/p/CXLk1XmpNUx/,https://scontent.cdninstagram.com/v/t51.29350-...,62,1,18014576203356847
113,2021-12-05T13:17:15+0000,https://www.instagram.com/p/CXGj9v4pFjs/,https://scontent.cdninstagram.com/v/t51.29350-...,44,0,18206637583136542
114,2021-12-01T11:55:03+0000,https://www.instagram.com/p/CW8HYTwpjgV/,https://scontent.cdninstagram.com/v/t51.29350-...,72,3,18272253838038998


In [6]:

'''ここで全投稿のmedia idのリストも作っておく'''

list_media_id = []
for noc in np.arange(len(result)):
    for nop in np.arange(len(result[noc])):
        media_id = result[noc][nop]['id']
        list_media_id.append(media_id)
print('最初の10個だけ表示：',list_media_id[:10])
print('全投稿数：',len(list_media_id))

最初の10個だけ表示： ['17992966069785815', '17974722560215754', '17998017016835519', '18356028925053534', '18056555143403921', '18358608613016133', '18083207689341792', '17987147557961392', '18279319102141506', '18019371583557456']
全投稿数： 116


In [7]:
'''投稿ごとのさらに詳細なinsight(reach, saved, impressions, enagegement)を取得する
    ※これは各投稿IDごとにAPIコールが必要なため、処理に少し時間がかかる。改善方法を検討中'''

"""
    ***********************************************************************************
    【APIエンドポイント】
    https://graph.facebook.com/{graph-api-version}/{ig-media-id}/insights?metric={metric}
    ***********************************************************************************
    """
#media IDからインサイトを取得する関数
def media_insight(media_id):
    
    '''リクエスト先のurl作成'''
    metric_for_url = ''
    for mt in metric:
        metric_for_url += mt + '%2C'
    metric_for_url = metric_for_url.rstrip('%2C')
    request_url = params['endpoint_base'] + media_id + '/insights?access_token=' + params[
        'access_token'] + '&metric=' + metric_for_url
    
    response = requests.get(request_url).json()['data']
    response_reshape = dict()
    response_reshape['id'] = media_id
    response_reshape['reach'] = response[0]['values'][0]['value']
    response_reshape['impressions'] = response[1]['values'][0]['value']
    response_reshape['saved'] = response[2]['values'][0]['value']
    response_reshape['engagement'] = response[3]['values'][0]['value']
    
    return response_reshape


metric = ['reach','impressions','saved','engagement']

#全てのmediaIDでインサイトを取得する
results = []
for nom in np.arange(len(list_media_id)):
    media_id = list_media_id[nom]
    out = media_insight(media_id)
    results.append(out)

In [8]:
df_media_insight = pd.DataFrame(results)
df_media_insight

,id,reach,impressions,saved,engagement
0,17992966069785815,171,179,3,63
1,17974722560215754,248,279,0,49
2,17998017016835519,199,219,1,57
3,18356028925053534,300,311,2,82
4,18056555143403921,247,263,1,64
...,...,...,...,...,...
111,17906032391509098,475,521,0,127
112,18014576203356847,465,500,0,63
113,18206637583136542,293,314,0,44
114,18272253838038998,334,366,0,75


In [9]:
'''結合してカラムの順番整理'''

df_media = pd.merge(df_media_info,df_media_insight, how='inner', on='id')
df_media = df_media.reindex(columns=[
    'id', 'timestamp','permalink','media_url','like_count','saved','reach','impressions','engagement'])
df_media

,id,timestamp,permalink,media_url,like_count,saved,reach,impressions,engagement
0,17992966069785815,2023-05-03T09:30:00+0000,https://www.instagram.com/p/CrvFzVnr1qb/,https://scontent.cdninstagram.com/v/t51.29350-...,59,3,171,179,63
1,17974722560215754,2023-05-02T09:30:00+0000,https://www.instagram.com/p/Cr0PXL2LIdk/,https://scontent.cdninstagram.com/v/t51.29350-...,49,0,248,279,49
2,17998017016835519,2023-05-01T09:30:00+0000,https://www.instagram.com/p/CrshBJzr1g9/,https://scontent.cdninstagram.com/v/t51.29350-...,56,1,199,219,57
3,18356028925053534,2023-04-27T09:30:00+0000,https://www.instagram.com/p/CrkymADrCAZ/,https://scontent.cdninstagram.com/v/t51.29350-...,80,2,300,311,82
4,18056555143403921,2023-04-26T09:30:00+0000,https://www.instagram.com/p/CrfpDdxLtal/,https://scontent.cdninstagram.com/v/t51.29350-...,63,1,247,263,64
...,...,...,...,...,...,...,...,...,...
111,17906032391509098,2022-01-13T13:16:58+0000,https://www.instagram.com/p/CYq-71wpmv4/,https://scontent.cdninstagram.com/v/t51.29350-...,127,0,475,521,127
112,18014576203356847,2021-12-07T12:01:02+0000,https://www.instagram.com/p/CXLk1XmpNUx/,https://scontent.cdninstagram.com/v/t51.29350-...,62,0,465,500,63
113,18206637583136542,2021-12-05T13:17:15+0000,https://www.instagram.com/p/CXGj9v4pFjs/,https://scontent.cdninstagram.com/v/t51.29350-...,44,0,293,314,44
114,18272253838038998,2021-12-01T11:55:03+0000,https://www.instagram.com/p/CW8HYTwpjgV/,https://scontent.cdninstagram.com/v/t51.29350-...,72,0,334,366,75


In [10]:
#csvに書き出して保存
today = datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')[:8]
df_media.to_csv('../data/output/media_insight_'+today+'.csv')

In [11]:
'''直近30日間のアカウントステータスを取得する'''
# ※30日は遡れる日数の上限

import datetime
#今日と28日さかのぼった日付をyyyy/mm/ddの形式で取得する 標準時に設定
delta_days = 30
today = datetime.datetime.utcnow()
date_delta = datetime.datetime.utcnow() - datetime.timedelta(days=delta_days)

yyyymmdd_td = '{yyyy}/{mm}/{dd}'.format(yyyy=today.year,mm=today.month,dd=today.day)
yyyymmdd_delta = '{yyyy}/{mm}/{dd}'.format(yyyy=date_delta.year,mm=date_delta.month,dd=date_delta.day)

print('今日の日付：',yyyymmdd_td)
print('遡った日付：',yyyymmdd_delta)

今日の日付： 2023/5/4
遡った日付： 2023/4/4


In [12]:
# エンドポイントURLの作成
metric = ['follower_count','impressions','profile_views','reach']
metric_for_url = ''
for mt in metric:
    metric_for_url += mt + '%2C'
metric_for_url = metric_for_url.rstrip('%2C')

request_url = params['endpoint_base'] + params['instagram_account_id'] + '/insights?access_token=' + params[
    'access_token'] + '&metric=' + metric_for_url + '&period=day'+ '&since=' + yyyymmdd_delta + '&until=' + yyyymmdd_td
response = requests.get(request_url).json()['data']

In [13]:
list_endtimes = []
list_impressions = []
list_follower_count = []
list_reach = []
list_profile_views = []

for day_n in range(delta_days):
    list_endtimes.append(response[0]['values'][day_n]['end_time'])
    list_follower_count.append(response[0]['values'][day_n]['value'])
    list_impressions.append(response[1]['values'][day_n]['value'])
    list_profile_views.append(response[2]['values'][day_n]['value'])
    list_reach.append(response[3]['values'][day_n]['value'])
    
result = dict()
result['endtime'] = list_endtimes
result['follower_count'] = list_follower_count
result['profile_views'] = list_profile_views
result['impressions'] = list_impressions
result['reach'] = list_reach


In [14]:
#データフレームとして格納
df_account_status_in30days = pd.DataFrame(result)
df_account_status_in30days

#時系列で並び替え
df_account_status_in30days = df_account_status_in30days.sort_values(by='endtime', ascending=False, ignore_index=True)

In [15]:
#csvに書き出して保存
today = datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')[:8]
df_account_status_in30days.to_csv('../data/output/account_status_'+today+'.csv')